In [25]:
from datasets import load_dataset
import datasets
import torchaudio
import numpy as np
import torch

In [14]:
dataset_cache_dir = "/home/sanchitgandhi/cache/huggingface/datasets"

In [15]:
swbd = load_dataset("/home/sanchitgandhi/datasets/datasets/switchboard", "switchboard", split="train[:1000]", cache_dir=dataset_cache_dir, use_auth_token=True, ignore_verifications=True)

Reusing dataset switchboard (/home/sanchitgandhi/cache/huggingface/datasets/switchboard/switchboard/1.1.0/dae178d6d13fe90241277e548542a56b7db74c8ce4d14acff28905b2cbb89f4a)


In [16]:
swbd

Dataset({
    features: ['audio', 'text', 'file', 'id'],
    num_rows: 1000
})

In [17]:
swbd_resampled = swbd.cast_column(
        "audio", datasets.features.Audio(sampling_rate=16_000)
    )

def prepare_dataset(batch):
    sample = batch["audio"]
    batch["resampled_inputs"] = sample["array"]
    batch["resampled_sampling_rate"] = sample["sampling_rate"]
    return batch

In [18]:
%%time

swbd_resampled = swbd_resampled.map(
        prepare_dataset,
        remove_columns=swbd_resampled.column_names,
        num_proc=1,
        desc="preprocess train dataset",
)

Loading cached processed dataset at /home/sanchitgandhi/cache/huggingface/datasets/switchboard/switchboard/1.1.0/dae178d6d13fe90241277e548542a56b7db74c8ce4d14acff28905b2cbb89f4a/cache-756e92b00c4436e3.arrow


CPU times: user 3.42 ms, sys: 3.82 ms, total: 7.24 ms
Wall time: 7.21 ms


In [28]:
resampler = torchaudio.transforms.Resample(8_000, 16_000)

def prepare_dataset(batch):
    sample = batch["audio"]["array"]
    speech_tensor = torch.FloatTensor(sample)
    speech_tensor = speech_tensor.squeeze()
    speech_tensor = resampler(speech_tensor)
    sampling_rate = resampler.new_freq
    
    batch["resampled_inputs"] = speech_tensor.numpy()
    batch["resampled_sampling_rate"] = sampling_rate
    return batch

In [29]:
%%time

swbd_resampled_torchaudio = swbd.map(
        prepare_dataset,
        remove_columns=swbd.column_names,
        num_proc=1,
        desc="preprocess train dataset",
)

preprocess train dataset: 100%|████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 197.88ex/s]


CPU times: user 2min 48s, sys: 2.39 s, total: 2min 51s
Wall time: 5.42 s


Librosa: 78s

torchaudio: 5.4s (14x faster)